In [1]:
# import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
# generate arrays of random latitude and longitude values
lat = np.random.uniform(-90.000,90.000, size=20)
lngs = np.random.uniform(-180.000,180.000, size=20)
lat_lngs = zip(lat, lngs)
lat_lngs

In [3]:
# add the latitudes and longitudes to a list. 
coordinates = list(lat_lngs)

In [4]:
# create a list for holding the cities
cities = []
# Identify the nearest city for each latitude and logitude combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list. 
    if city not in cities:
        cities.append(city)
# print the city count to confirm sufficient count:
len(cities)

20

In [5]:
# import requests and api_key
import requests
from config import weather_api_key

In [6]:
# starting URL for weather map API call:
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&lat={lat}&lon={lon}&appid=" + weather_api_key

In [7]:
# import the time lib and the datetime module 
import time
from datetime import datetime

In [8]:
coordinates

[(-41.23452104259156, 151.1316017820236),
 (-1.7783775105044697, 55.17846577383608),
 (10.639271197364664, 149.986944012239),
 (-67.64288568322307, 163.20682685280178),
 (79.2612531870409, 50.21693093612308),
 (-53.35262086326482, 121.99051853814524),
 (23.698991075375204, -156.4993584031517),
 (-0.9559704781441098, -5.005064885248544),
 (25.469925846905227, -137.536085493628),
 (35.05616210730096, 100.25953753349614),
 (88.84605231037307, -116.20586586407964),
 (77.71452373346492, -73.37593097435153),
 (15.352313618737938, -0.25045839042138596),
 (17.27311909395594, -48.29521080581094),
 (33.55010074542923, -74.89360747215053),
 (-50.621250376533396, 44.086794210214094),
 (25.532693271547387, -164.80602856606532),
 (-26.445398569305226, -157.8225353393986),
 (31.937416819531308, 61.86139984887939),
 (69.13458592304545, 69.86674333862246)]

In [9]:
# create an empty list to hold the weather data
city_data = []

# logging statements
print("Biginning Data Retrieval     ")
print("-----------------------------")

# create counters
record_count = 1
set_count = 1

# loop through all the cities in the list
for i, city in enumerate(cities):
    
    # group citiesin sets of 50 for logging purposes
    if (i % 10 == 0 and i>= 10):
        set_count += 1
        record_count = 1
        # pause the api for 60 seconds
#         time.sleep(60)

    # create endpoint url with each city, while removing spaces in the city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # log the url, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    
    # run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        #convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime("%Y-%m-%d %H:%M:%S")

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                              "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              "Wind Speed": city_wind,
                              "Current Description": city_description,
                              "Country": city_country,
                              "Date": city_date})

    #If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass


# Indicate that data loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Biginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | launceston
Processing Record 2 of Set 1 | victoria
Processing Record 3 of Set 1 | lorengau
Processing Record 4 of Set 1 | bluff
Processing Record 5 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 6 of Set 1 | albany
Processing Record 7 of Set 1 | kailua
Processing Record 8 of Set 1 | tabou
Processing Record 9 of Set 1 | hilo
Processing Record 10 of Set 1 | xining
Processing Record 1 of Set 2 | tuktoyaktuk
Processing Record 2 of Set 2 | qaanaaq
Processing Record 3 of Set 2 | gorom-gorom
Processing Record 4 of Set 2 | bathsheba
Processing Record 5 of Set 2 | havelock
Processing Record 6 of Set 2 | east london
Processing Record 7 of Set 2 | kapaa
Processing Record 8 of Set 2 | avarua
Processing Record 9 of Set 2 | farah
Processing Record 10 of Set 2 | yar-sale
-----------------------------
Data Retrieval Complete      
-----------------------------


In [15]:
# count the number of cities collected
print(len(city_data))

19


In [20]:
# convert the array of dictionaries into a pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,Launceston,-41.4500,147.1667,60.62,95,40,3.44,moderate rain,AU,2022-04-18 00:06:35
1,Victoria,22.2855,114.1577,71.65,80,100,4.00,overcast clouds,HK,2022-04-18 00:04:48
2,Lorengau,-2.0226,147.2712,83.08,79,61,6.93,broken clouds,PG,2022-04-18 00:07:05
3,Bluff,-46.6000,168.3333,60.80,67,11,5.39,few clouds,NZ,2022-04-18 00:00:44
4,Albany,42.6001,-73.9662,42.64,57,59,3.00,broken clouds,US,2022-04-17 23:57:05
5,Kailua,21.4022,-157.7394,82.38,74,18,8.01,few clouds,US,2022-04-18 00:03:46
6,Tabou,4.4230,-7.3528,77.05,84,100,2.71,overcast clouds,CI,2022-04-18 00:07:06
7,Hilo,19.7297,-155.0900,77.29,88,100,3.44,light rain,US,2022-04-18 00:07:07
8,Xining,36.6167,101.7667,34.95,50,47,1.88,scattered clouds,CN,2022-04-18 00:07:07
9,Tuktoyaktuk,69.4541,-133.0374,15.49,83,99,8.61,overcast clouds,CA,2022-04-18 00:00:54


In [23]:
# Reorder the columns
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed","Current Description"]

# Assign the new column order to the DF
city_data_df = city_data_df[new_column_order]

In [24]:
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Launceston,AU,-41.4500,147.1667,60.62,95,40,3.44,moderate rain
1,Victoria,HK,22.2855,114.1577,71.65,80,100,4.00,overcast clouds
2,Lorengau,PG,-2.0226,147.2712,83.08,79,61,6.93,broken clouds
3,Bluff,NZ,-46.6000,168.3333,60.80,67,11,5.39,few clouds
4,Albany,US,42.6001,-73.9662,42.64,57,59,3.00,broken clouds
5,Kailua,US,21.4022,-157.7394,82.38,74,18,8.01,few clouds
6,Tabou,CI,4.4230,-7.3528,77.05,84,100,2.71,overcast clouds
7,Hilo,US,19.7297,-155.0900,77.29,88,100,3.44,light rain
8,Xining,CN,36.6167,101.7667,34.95,50,47,1.88,scattered clouds
9,Tuktoyaktuk,CA,69.4541,-133.0374,15.49,83,99,8.61,overcast clouds
